In [1]:
!pip install folium

     |████████████████████████████████| 92kB 8.0MB/s eta 0:00:011


In [2]:
import pandas as pd
import folium

In [3]:
CLIENT_ID = 'PRNGL0VL3ECZKTXEISLT0VLGJTYMJUIQAHA3SGOFTIYMHJKT' # your Foursquare ID
CLIENT_SECRET = 'CVCNL0UQTYNDOQESA22FPWHTY0JKVLE4QKRMZXKJ33JSQRMT' # your Foursquare Secret
VERSION = '20200309' # Foursquare API version

In [4]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20200309',
    "section": "coffee",
    "near": "Puducherry",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [5]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [6]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Puducherry', 'Puducherry')

In [7]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 11.940617444027888, 'lng': 79.83624478198776},
  'sw': {'lat': 11.929343550284385, 'lng': 79.829891578257}},
 10)

In [8]:
d["geocode"]

{'what': '',
 'where': 'puducherry',
 'center': {'lat': 11.93381, 'lng': 79.82979},
 'displayString': 'Puducherry, Union Territory of Puducherry, India',
 'cc': 'IN',
 'geometry': {'bounds': {'ne': {'lat': 11.989259814081308,
    'lng': 79.85232329825784},
   'sw': {'lat': 11.78841565732688, 'lng': 79.75705811820067}}},
 'slug': 'puducherry-india',
 'longId': '72057594039187361'}

In [9]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [10]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [11]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 10


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '50d0323de4b0bcb85621bd31',
  'name': 'ZUKA Choco-la',
  'location': {'address': 'No 319 Mission St',
   'lat': 11.929856,
   'lng': 79.830246,
   'labeledLatLngs': [{'label': 'display',
     'lat': 11.929856,
     'lng': 79.830246}],
   'cc': 'IN',
   'country': 'India',
   'formattedAddress': ['No 319 Mission St', 'India']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-50d0323de4b0bcb85621bd31-0'}

In [12]:
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '502d161ce4b0504d9d3d2532',
  'name': 'The Indian Kaffe Express',
  'location': {'address': 'No. 3 Dumas St.',
   'crossStreet': 'Opp. Dumas Church',
   'lat': 11.930366734017877,
   'lng': 79.83478365184189,
   'labeledLatLngs': [{'label': 'display',
     'lat': 11.930366734017877,
     'lng': 79.83478365184189}],
   'postalCode': '605001',
   'cc': 'IN',
   'city': 'Puducherry',
   'state': 'Union Territory of Puducherry',
   'country': 'India',
   'formattedAddress': ['No. 3 Dumas St. (Opp. Dumas Church)',
    'Puducherry 605001',
    'Union Territory of Puducherry',
    'India']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png

In [13]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["formattedAddress"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "formattedAddress", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 3 cafes


,uid,name,shortname,formattedAddress,postalcode,lat,lng
0,502d161ce4b0504d9d3d2532,The Indian Kaffe Express,Coffee Shop,"[No. 3 Dumas St. (Opp. Dumas Church), Puducher...",605001,11.930367,79.834784
1,51c6e60b498ea9955437d414,KBS Kofi Bar (Beach Branch),Coffee Shop,"[1 (Rue Saint Martin), Pondicherry 605001, Pud...",605001,11.935574,79.835956
2,5a4e4b2b2105c31a50a3c1eb,Coromandel Cafe,Café,"[Puducherry 605001, Union Territory of Puduche...",605001,11.931295,79.834144


In [18]:
pdy_center = d["geocode"]["center"]
pdy_center

{'lat': 11.93381, 'lng': 79.82979}

In [19]:
from folium import plugins

# create map of Istanbul using latitude and longitude values
map_pdy = folium.Map(location=[pdy_center["lat"], pdy_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_pdy)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_pdy.add_child(plugins.HeatMap(hm_data))

map_pdy

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [20]:
lat = 11.93381 
lng = 79.82979
map_pdy = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="CP's!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_pdy)
map_pdy